# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [4]:
uk_emb = KeyedVectors.load_word2vec_format("/content/drive/My Drive/ml/cc.uk.300.vec")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ru_emb = KeyedVectors.load_word2vec_format("/content/drive/My Drive/ml/cc.ru.300.vec")

In [6]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [9]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [ ]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2021-10-15 08:49:51--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2021-10-15 08:49:52--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-10-15 08:49:52 ERROR 404: Not Found.



In [ ]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

In [10]:
uk_ru_train, X_train, Y_train = load_word_pairs("/content/drive/My Drive/ml/ukr_rus.train.txt")

In [11]:
uk_ru_test, X_test, Y_test = load_word_pairs("/content/drive/My Drive/ml/ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [12]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)
# -------

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [13]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531432747840881),
 ('июнь', 0.8402522802352905),
 ('март', 0.8385884165763855),
 ('сентябрь', 0.8331484794616699),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278019428253174),
 ('ноябрь', 0.8243728280067444),
 ('июль', 0.8229618072509766),
 ('август', 0.8112280368804932),
 ('январь', 0.8022986650466919)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [14]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        num_matches += int(ru in [j[0] for j in ru_emb.most_similar([mapped_vectors[i]], topn=topn)])
    precision_val = num_matches / len(pairs)
    return precision_val

In [15]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [16]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [17]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [18]:
print(precision_top1)
print(precision_top5)

0.628498727735369
0.7913486005089059


**Conclusion:** for top1 this method isn't very accurate.

## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [19]:
import numpy as np

In [20]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    u, s, vh = np.linalg.svd(X_train.T @ Y_train)
    mapping = u @ vh

    return mapping

In [21]:
W = learn_transform(X_train, Y_train)

In [22]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.805662989616394),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938188910484314),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

In [23]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


**Conclusion:** accuracy hasn't improved very much.

## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [24]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2021-10-17 15:02:30--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  7.90MB/s    in 0.2s    

2021-10-17 15:02:30 (7.90 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [25]:
!gzip -d ./uk.txt.gz

In [26]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [27]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [28]:
# Any necessary preprocessing if needed

uk_corpus[:30]

['Я вже закінчу коледж, коли ви вернетеся з Америки.\n',
 'Він наказав мені негайно вийти з кімнати.\n',
 'Як би ти не намагався, ти не вивчиш англійську за два-три місяці.\n',
 'Поки я не подзвонив, він не прийшов.\n',
 'У всесвіті багато галактик.\n',
 'Вона приймає душ щоранку.\n',
 'Неслухняний хлопчик заблукав й оглядався по сторонах.\n',
 'Вона повільно зникала в туманному лісі.\n',
 'Наш літак летів понад хмарами.\n',
 'У Майка є декілька друзів у Флориді.\n',
 'Місто бомбардували ворожі літаки.\n',
 'Я зустрінуся з тобою в неділю о третій.\n',
 'Для фінансування війни було видано облігації.\n',
 'Звідки беруть початок Олімпійські ігри?\n',
 'Я сказав собі: «Це гарна ідея».\n',
 'Ми збиралися пробути там біля двох тижнів.\n',
 'Як на мене, то наразі помовчу.\n',
 'Мій дядько вчора помер від раку шлунку.\n',
 'Здається, діти втомилися від плавання.\n',
 'Немає кохання без ревнощів.\n',
 'Можливо, я антисоціальний, але це не означає, що я не спілкуюся з людьми.\n',
 'Я не знаю, що

In [29]:
import re

def str2words(text):
    text = text.lower()
    #text = re.sub(r'[^ґїієа-я\s\d]', '', text)
    text = re.sub('\\n', '', text)
    text = re.findall(r"[ґїієа-я]+|\S", text)
    return text

print(str2words(uk_corpus[0]))

['я', 'вже', 'закінчу', 'коледж', ',', 'коли', 'ви', 'вернетеся', 'з', 'америки', '.']


In [30]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    words = str2words(sentence)
    translated = []
    for word in words:
        if word in uk_emb:
            translated.append(ru_emb.most_similar([np.matmul(uk_emb[word], W)], topn=1)[0][0])
        else:
            translated.append('UNK')
    return " ".join(translated)

In [32]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [58]:
for sent in uk_corpus[::10]:
    print('uk: ', sent[:-1])
    print('ru: ', translate(sent), '\n')

uk:  Я вже закінчу коледж, коли ви вернетеся з Америки.
ru:  мной уже закончу колледж , когда мы прибежишь со америки . 

uk:  Місто бомбардували ворожі літаки.
ru:  город бомбили враждебные самолеты . 

uk:  Можливо, я антисоціальний, але це не означає, що я не спілкуюся з людьми.
ru:  возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми . 

uk:  Цього ранку випала роса.
ru:  этого утра выпала роса . 

uk:  Біда не приходить одна.
ru:  беда не приходит одна . 

uk:  Подивися на той дим.
ru:  посмотри по тот дым . 

uk:  Я замовив два гамбургера.
ru:  мной заказал два гамбургера . 

uk:  Я не хотів нікого образити.
ru:  мной не хотел никого обидеть . 

uk:  Гора вкрита снігом.
ru:  гора покрыта снегом . 

uk:  На фотографії в дівчини корона не з золота, а з квітів.
ru:  по фотографии во девушки корона не со золота , а со цветов . 

uk:  У мене є мрія.
ru:  во меня То мечта . 

uk:  Я приїхав у Японію з Китаю.
ru:  мной приехал во UNK со китая . 

uk:

Ok, let's see how many there are out-of-vocabulary words.

In [33]:
ooc = set(filter(lambda x: x not in uk_emb, str2words(' '.join(uk_corpus))))


In [34]:
print('Count of OOC words: ', len(ooc))
print(ooc)

Count of OOC words:  79
{'голландії', 'прокомпостуйте', 'орлеані', 'вітвер', 'шекспіра', 'рейн', 'поїдзка', 'савако', 'неевклідовий', 'британію', 'індонезії', 'хіросі', 'товада', 'відремонтуй', 'ганусина', 'найабсурдніша', 'позбігнути', 'спростуваті', 'іспанією', 'минилу', 'дівчіна', 'рушді', 'нідерландами', 'атланті', 'італією', 'амфітрити', 'гегеля', 'тед', 'айріс', 'подобавляти', 'чікаґо', 'розхльобуй', 'пікассо', 'дударева', 'юбілеєм', 'терезою', 'олексія', 'джорджія', 'мехіко', 'гартман', 'флориді', 'трасянку', 'кеном', 'мюріел', 'кіото', 'коштовнішого', 'шанхаї', 'венеція', 'примірю', 'краплистою', 'мідорі', 'англійськіх', 'торонто', 'нюношку', 'уельсі', 'хіроші', 'зневолення', 'китаєм', 'танабата', 'вчитимуся', 'діани', 'ятьдесять', 'німеччиною', 'міллера', 'сьюзен', 'нюношка', 'лондону', 'японію', 'маюко', 'пекін', 'салмана', 'кюсю', 'скучатиму', 'прибиральня', 'платон', 'удамо', 'ниряємо', 'шотландія', 'мініспідниці'}


Not great, not terrible.
One of the possible approaches is to compose a word vector from vectors for its subwords. For example, word <*grey*> is consists of <*gr, gre, rey, ey*>.  Then, in addition to the vector for this word, we also use vectors for character n-grams (which are also in the vocabulary).

This changes only the way we form word vector; the whole training pipeline is the same as in the standard Word2Vec. 

This implemented in fasttext library.

In [50]:
pip install fasttext

     |████████████████████████████████| 68 kB 4.4 MB/s 
  Using cached pybind11-2.8.0-py2.py3-none-any.whl (207 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3126415 sha256=31b4880e47d74f86701b8ca79c4dbf42665cbfab406bf8257a474dbdbd426502
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [51]:
import fasttext
ft = fasttext.load_model("/content/drive/My Drive/ml/cc.uk.300.bin")

Let's see on the example the difference between word2vec and fasttext. The word **голландії** is not in the vocabulary.

In [53]:
ru_emb.most_similar([np.matmul(ft['голландії'], W)], topn = 1)[0][0]

'голландии'

In [54]:
ru_emb.most_similar([np.matmul(uk_emb['голландії'], W)], topn = 1)[0][0]

KeyError: ignored

In [55]:
def better_translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    words = str2words(sentence)
    translated = []
    for word in words:
        translated.append(ru_emb.most_similar([np.matmul(ft[word], W)], topn=1)[0][0])
    return " ".join(translated)

In [57]:
for sent in uk_corpus[::10]:
    print('uk: ', sent[:-1])
    print('ru: ', better_translate(sent), '\n')

uk:  Я вже закінчу коледж, коли ви вернетеся з Америки.
ru:  мной уже закончу колледж , когда мы прибежишь со америки . 

uk:  Місто бомбардували ворожі літаки.
ru:  город бомбили враждебные самолеты . 

uk:  Можливо, я антисоціальний, але це не означає, що я не спілкуюся з людьми.
ru:  возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми . 

uk:  Цього ранку випала роса.
ru:  этого утра выпала роса . 

uk:  Біда не приходить одна.
ru:  беда не приходит одна . 

uk:  Подивися на той дим.
ru:  посмотри по тот дым . 

uk:  Я замовив два гамбургера.
ru:  мной заказал два гамбургера . 

uk:  Я не хотів нікого образити.
ru:  мной не хотел никого обидеть . 

uk:  Гора вкрита снігом.
ru:  гора покрыта снегом . 

uk:  На фотографії в дівчини корона не з золота, а з квітів.
ru:  по фотографии во девушки корона не со золота , а со цветов . 

uk:  У мене є мрія.
ru:  во меня То мечта . 

uk:  Я приїхав у Японію з Китаю.
ru:  мной приехал во австралию со китая .

Great! 
See second notebook for the Neural Machine Translation assignment.

**Conclusion:** for such a simple translator the result isn't as bad as it seems. However there're a lot of flaws. Not accurate translation, if we use only top1, and the problem with out-of-vocabulary words, which can be solved by using fasttext  instead of word2vectors.